# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [18]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import json
import plotly.express as px

# Import API key
from api_keys import geoapify_key

In [19]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,qaanaaq,77.4840,-69.3632,-16.32,66,88,3.26,GL,1675396695
1,1,rikitea,-23.1203,-134.9692,26.30,71,36,6.62,PF,1675396695
2,2,aykhal,66.0000,111.5000,-25.50,96,100,2.49,RU,1675396696
3,3,ushuaia,-54.8000,-68.3000,12.81,62,75,1.03,AR,1675396585
4,4,faya,18.3851,42.4509,7.06,50,32,1.60,SA,1675396696


In [3]:
#Convert to Farenheit
city_data_df['Max Temp'] = city_data_df['Max Temp'].apply(lambda x: (x * 9/5) + 32)

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,qaanaaq,77.4840,-69.3632,2.624,66,88,3.26,GL,1675396695
1,1,rikitea,-23.1203,-134.9692,79.340,71,36,6.62,PF,1675396695
2,2,aykhal,66.0000,111.5000,-13.900,96,100,2.49,RU,1675396696
3,3,ushuaia,-54.8000,-68.3000,55.058,62,75,1.03,AR,1675396585
4,4,faya,18.3851,42.4509,44.708,50,32,1.60,SA,1675396696


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:
%%capture --no-display

# Configure the map plot
city_data_map = city_data_df.hvplot.points(
    x="Lng",
    y="Lat",
    size="Humidity",
    hover_cols=["City","Country"],
    color="blue",
    title="Cities Latitude and Longitude vs. Humidity",
)

# Display the map
city_data_map

:Points   [Lng,Lat]   (Humidity,City,Country)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
# Narrow down cities that fit criteria and drop any results with null values
city_data_df = city_data_df[(city_data_df['Max Temp'] > 65) & (city_data_df['Max Temp'] < 85) & (city_data_df['Wind Speed'] < 10)]

# Drop any rows with null values
city_data_df = city_data_df.dropna()

# Display sample data
city_data_df.head()


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
1,1,rikitea,-23.1203,-134.9692,79.340,71,36,6.62,PF,1675396695
7,7,mudgee,-32.5943,149.5871,74.786,23,0,6.45,AU,1675396697
10,10,new norfolk,-42.7826,147.0587,68.720,41,48,0.89,AU,1675396698
12,12,hambantota,6.1241,81.1185,79.142,100,75,4.12,LK,1675396699
13,13,port alfred,-33.5906,26.8910,66.182,90,62,2.24,ZA,1675396699


### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
1,rikitea,PF,-23.1203,-134.9692,71,
7,mudgee,AU,-32.5943,149.5871,23,
10,new norfolk,AU,-42.7826,147.0587,41,
12,hambantota,LK,6.1241,81.1185,100,
13,port alfred,ZA,-33.5906,26.8910,90,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [21]:
# Set parameters to search for a hotel
radius = 10000 # in meters
params = {
"radius": radius,
"q": "accomodation.hotel"
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"withinCircle({longitude},{latitude},{radius})"
    params["bias"] = f"nearest({longitude},{latitude})"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make an API request using the params dictionaty
    name_address = requests.get(base_url, params=params).json()
    
    # Convert the API response to JSON format
    name_address = json.loads(name_address)
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
rikitea - nearest hotel: No hotel found
mudgee - nearest hotel: No hotel found
new norfolk - nearest hotel: No hotel found
hambantota - nearest hotel: No hotel found
port alfred - nearest hotel: No hotel found
atuona - nearest hotel: No hotel found
saint-philippe - nearest hotel: No hotel found
east london - nearest hotel: No hotel found
bathsheba - nearest hotel: No hotel found
puerto ayora - nearest hotel: No hotel found
ulladulla - nearest hotel: No hotel found
kapaa - nearest hotel: No hotel found
cabedelo - nearest hotel: No hotel found
caravelas - nearest hotel: No hotel found
carutapera - nearest hotel: No hotel found
victoria - nearest hotel: No hotel found
port elizabeth - nearest hotel: No hotel found
pocone - nearest hotel: No hotel found
shakawe - nearest hotel: No hotel found
souillac - nearest hotel: No hotel found
charters towers - nearest hotel: No hotel found
tecoanapa - nearest hotel: No hotel found
hobart - nearest hotel: No hotel found
busselto

,City,Country,Lat,Lng,Humidity,Hotel Name
1,rikitea,PF,-23.1203,-134.9692,71,No hotel found
7,mudgee,AU,-32.5943,149.5871,23,No hotel found
10,new norfolk,AU,-42.7826,147.0587,41,No hotel found
12,hambantota,LK,6.1241,81.1185,100,No hotel found
13,port alfred,ZA,-33.5906,26.8910,90,No hotel found
...,...,...,...,...,...,...
539,gizo,SB,-8.1030,156.8419,73,No hotel found
542,mogadishu,SO,2.0371,45.3438,85,No hotel found
543,hassan,IN,13.0006,76.0994,37,No hotel found
549,jacqueville,CI,5.2052,-4.4146,86,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [20]:
%%capture --no-display

# Configure the map plot
fig = px.scatter_mapbox(hotel_df, lat="Lat", lon="Lng", text="City", color="City", zoom=5)

fig.update_layout(mapbox_style="open-street-map", title="Cities with Accommodation")

# Display the map
fig.show()